In [149]:
import math
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import KFold


In [150]:
train_in = np.loadtxt('./Datasets/train_in.csv', 
                  delimiter=',',  
                  skiprows=0,      
                  dtype=float)       
test_in = np.loadtxt('./Datasets/test_in.csv', 
                  delimiter=',',   
                  skiprows=0,      
                  dtype=float)       
train_out = np.loadtxt('./Datasets/train_out.csv', 
                  delimiter=',',  
                  skiprows=0,      
                  dtype=float)       
test_out = np.loadtxt('./Datasets/test_out.csv', 
                  delimiter=',',   
                  skiprows=0,      
                  dtype=float)       

In [151]:
X_train, X_val, y_train, y_val = train_test_split(train_in, train_out, train_size=0.8, random_state=0)
X_test = test_in
y_test = test_out
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(1365, 256) (342, 256) (1365,) (342,)


In [152]:
# input dataset
def preprocessing(datasets):
    return np.hstack((datasets, np.ones(len(datasets)).reshape(len(datasets),1)))

In [153]:
def get_dummies(list, array):
    for i, each in zip(list, array):
        each[int(i)] = 1
    return array

In [154]:
# sigmoid function 
def sig(x):
    return (1/(1+np.exp(-x)))

def sig_prime(x):
    return (x * (1 - x))

In [155]:
Len_train = len(X_train)
Len_val = len(X_val)
Len_test = len(X_test)

X_train = preprocessing(X_train)
X_val = preprocessing(X_val)
X_test = preprocessing(X_test)

y_train_dummy = np.zeros(Len_train*10).reshape(Len_train, 10)
y_train_dummy = get_dummies(y_train, y_train_dummy)
y_val_dummy = np.zeros(Len_val*10).reshape(Len_val, 10)
y_val_dummy = get_dummies(y_val, y_val_dummy)
y_test_dummy = np.zeros(Len_test*10).reshape(Len_test, 10)
y_test_dummy = get_dummies(y_test, y_test_dummy)

In [156]:
kf = KFold(n_splits=5)

kfold_index_list = []

for cv_index_train, cv_index_test in kf.split(X_train):
    kfold_index_list.append((cv_index_train, cv_index_test))

In [157]:
def training(lr, w, x, y_dummy):
    for iter in range(1000):
        for i in range(len(x)):
            y_hat = sig(np.dot(x[i], w))
            diff = y_dummy[i] - y_hat
            x_re = x[i].reshape(257,1)
            # print(diff)
            sp = sig_prime(y_hat) * diff
            w += lr * x_re * sp

        if iter % 10 == 0 :
            arg_y_hat = np.argmax(sig(np.dot(x, w)), axis=1)
            arg_y_dummy = np.argmax(y_dummy, axis=1)
            arg_diff = abs(arg_y_dummy - arg_y_hat)
            acc = (len(y_dummy)-np.sum(arg_diff>0))/len(y_dummy)
            print(acc)
        if acc>0.97:
            break
    return w

In [158]:
np.random.seed(2022)
w0 = np.zeros(257*10).reshape(257,10)
lr = 0.3
w = training(lr, w0, X_train, y_train_dummy)

0.8534798534798534
0.9626373626373627
0.9663003663003663
0.9684981684981685
0.96996336996337
0.9706959706959707


In [159]:
# def training(lr, w, train_x, y_train_dummy):
#     for iter in range(100000):
#         y_hat = sig(np.dot(train_x, w))
#         # e = pow((y_train_dummy - y) * (y_train_dummy - y), 0.5)
#         diff = y_train_dummy - y_hat
#         # if iter % 100 == 0 :
#         #    print(sum(sum(abs(e))))
#         if sum(sum(abs(diff))) <= 1e-04:
#             # print(iter)
#             break
#         # updata w0(weight)
#         # print(w0.shape, train_x.shape, e.shape)
#         # print((1-w0) * w0)
#         # w0 += lr * np.dot(train_x.T, e) * ((1-w0) * w0)
#         w += lr * np.dot(train_x.T, diff)
#         # print(sum(sum(((1-w0) * w0))))
#     return w

In [160]:
def predict(lr, w, train_x, y_train_dummy):
    y_hat = sig(np.dot(train_x, w))
    return np.argmax(y_hat, axis=1)

In [161]:
np.random.seed(2022)
w0 = np.random.uniform(-0.3,0.3,size=(257,10))

lr = 0.3
# w = train(lr, w0, train_x, y_train_dummy)
results = []
w_list = []

for i in range(5):
    w = training(lr, w0, X_train[kfold_index_list[i][0]], y_train_dummy[kfold_index_list[i][0]])

    pred_train = predict(lr, w, X_train[kfold_index_list[i][1]], y_train_dummy[kfold_index_list[i][1]])
    acc_train = accuracy_score(y_train[kfold_index_list[i][1]], pred_train)

    pred_val = predict(lr, w, X_val, y_val_dummy)
    acc_val = accuracy_score(y_val, pred_val)

    pred_test = predict(lr, w, X_test, y_test_dummy)
    acc_test = accuracy_score(y_test, pred_test)

    w_list.append(w)
    results.append((acc_train, acc_val, acc_test))


0.7371794871794872
0.9578754578754579
0.9642857142857143
0.967032967032967
0.967032967032967
0.9688644688644689
0.9706959706959707
0.967948717948718
0.9706959706959707
0.967032967032967
0.9688644688644689
0.9706959706959707
0.967948717948718
0.967948717948718
0.9688644688644689
0.9688644688644689
0.9688644688644689
0.9697802197802198
0.9697802197802198
0.9697802197802198
0.9697802197802198
0.9716117216117216
0.9706959706959707


In [162]:
for acc, acc_val, acc_test in results:
    print("Accuracy train: ", acc, "Accuracy validation", acc_val, "Accuracy test", acc_test)

Accuracy train:  0.9413919413919414 Accuracy validation 0.9502923976608187 Accuracy test 0.872
Accuracy train:  0.9560439560439561 Accuracy validation 0.9473684210526315 Accuracy test 0.882
Accuracy train:  0.9706959706959707 Accuracy validation 0.956140350877193 Accuracy test 0.883
Accuracy train:  0.9816849816849816 Accuracy validation 0.9473684210526315 Accuracy test 0.881
Accuracy train:  0.9816849816849816 Accuracy validation 0.9502923976608187 Accuracy test 0.879


In [163]:
## confusion Test
confusion_matrix(predict(lr, w_list[1], X_test, y_test_dummy), y_test)

array([[211,   0,   0,   2,   1,   3,   2,   0,   3,   0],
       [  0, 114,   0,   0,   1,   0,   0,   0,   0,   1],
       [  4,   1,  83,   4,   1,   0,   0,   1,   1,   0],
       [  2,   0,   2,  65,   0,   9,   1,   1,   4,   0],
       [  3,   2,   4,   0,  73,   1,   1,   3,   1,   3],
       [  0,   0,   0,   1,   2,  37,   0,   0,   2,   1],
       [  2,   2,   2,   0,   2,   1,  85,   0,   0,   0],
       [  0,   1,   1,   1,   4,   2,   0,  56,   3,   4],
       [  1,   0,   9,   5,   0,   0,   1,   0,  78,   2],
       [  1,   1,   0,   1,   2,   2,   0,   3,   0,  77]])

In [164]:
# w0 = np.random.random((257,10))
# lr = 0.3
# w = training(lr, w0, X_train, y_train_dummy)